In [1]:
import face_recognition
import numpy as np
import dlib
from imutils import face_utils
import os
import cv2

os.environ["OPENCV_VIDEOIO_PRIORITY_MSMF"] = "0"

In [2]:
from keras.models import load_model

multi_model = load_model("weights.28-3.73.hdf5")
emodel=load_model("model_v6_23.hdf5")

Using TensorFlow backend.


In [66]:
Known_face_encodings = []
known_face_names = []
path="./facedataset/"
for image_db in os.listdir(path):
    Known_face_encodings.append(face_recognition.face_encodings(face_recognition.load_image_file(path+image_db))[0])
    known_face_names.append(image_db.split('.')[0])

In [93]:
emotion_dict= {0:'Angry',5:'Neutral', 4:'Neutral',1:'Neutral',6:'Surprise',2:'Neutral',3:'Happy'}
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
cap = cv2.VideoCapture('howdy.mp4')
writer = cv2.VideoWriter("o.avi",cv2.VideoWriter_fourcc(*"MJPG"), 30,(int(cap.get(3)),int(cap.get(4)))) 

while (cap.isOpened()):
    
    ret, frame=cap.read()
    if not ret:
        break  
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)  
    subjects= detect(gray, 1)
    
    for subject in subjects: 
        shape = predict(gray, subject)
        shape = face_utils.shape_to_np(shape)
        (x, y, w, h) = face_utils.rect_to_bb(subject)

        if frame[y-50:y+h+20,x-50:x+w+20].size!=0:
            roi_color=frame[y-50:y+h+20,x-50:x+w+20]
            roi_gray=gray[y-50:y+h+20,x-50:x+w+20]
        else :
            roi_color=frame[y:y+h,x:x+w]
            roi_gray=gray[y:y+h,x:x+w] 
                
        
        result=multi_model.predict(cv2.resize(roi_color,(64,64)).reshape(-1,64,64,3))
        eresult=np.argmax(emodel.predict(cv2.resize(roi_gray,(48,48)).reshape(-1,48,48,1)))

        label = "{0} {1} {2}".format(int(np.argmax(result[1],axis=1)[0]+1),"M" if np.argmax(result[0],axis=1)[0]==1 else "F",emotion_dict[eresult] if eresult in sorted(emotion_dict)  else  ' ')       
        
        label_name="Unkown"
        
        try:
            unknow_encoding = face_recognition.face_encodings(roi_color)[0]
        except:
            pass
        else:
            face_distances = face_recognition.face_distance(Known_face_encodings, unknow_encoding)            
            if len(face_distances[np.where(face_distances<0.5)])!=0:
                label_name=known_face_names[np.argmin(face_distances)]  

            

        cv2.rectangle(frame,(x-20,y-50),(x+w+25,y+h+50),(0,0,255),4) 
        cv2.rectangle(frame,(x-20,y+h),(x+w+25,y+h+50),(0,0,255),-1)
        
        cv2.putText(frame,label_name, (x-20,y+h+20), cv2.FONT_HERSHEY_PLAIN,1, (0,255,255), 2)
        cv2.putText(frame,label, (x-20,y+h+40),cv2.FONT_HERSHEY_PLAIN,1, (0,255,255), 2)

    writer.write(frame)    
    cv2.imshow('Face recognition with Age Gender Emotion Dectection', frame)
        
    if cv2.waitKey(1)==13:
        break
    
writer.release()
    
cap.release()            
            
cv2.destroyAllWindows()       